# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [15]:
# import libraries
import pysftp # library provides sftp connectivity tools
import pandas as pd # library provides mathematical suit 
from datetime import datetime as dt #library allows for manipulation of dates and time
import os # library allows for detection and manipulation of file paths / directories
from sqlalchemy import create_engine # library allows for creation of sql engine


In [16]:
# load messages dataset
messages = pd.read_csv('C:/Users/brian.meki/Desktop/Udacity/Data/disaster_messages.csv')

# clean data by removing duplicated rows
messages = messages.drop_duplicates('id')
print(messages.shape[0],
messages.head())

26180    id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12  says: west side of Haiti, rest of the country ...   

                                            original   genre  
0  Un front froid se retrouve sur Cuba ce matin. ...  direct  
1                 Cyclone nan fini osinon li pa fini  direct  
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct  
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct  
4  facade ouest d Haiti et le reste du pays aujou...  direct  


In [17]:
# load categories dataset
categories =  pd.read_csv('C:/Users/brian.meki/Desktop/Udacity/Data/disaster_categories.csv')

# clean data by removing duplicated rows
categories = categories.drop_duplicates('id')

print(categories.shape[0],
categories.head())



26180    id                                         categories
0   2  related-1;request-0;offer-0;aid_related-0;medi...
1   7  related-1;request-0;offer-0;aid_related-1;medi...
2   8  related-1;request-0;offer-0;aid_related-0;medi...
3   9  related-1;request-1;offer-0;aid_related-1;medi...
4  12  related-1;request-0;offer-0;aid_related-0;medi...


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [18]:
# merge datasets
df_Initial = pd.merge(messages, categories, on='id', how='inner')

print(df_Initial.duplicated('id').sum(),
      df_Initial.shape[0])

0 26180


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [19]:
# create a dataframe of the 36 individual category columns
categories = categories['categories'].str.split(';', expand=True)
categories.head();

In [20]:
# select the first row of the categories dataframe
row = categories.iloc[0]

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = [value[:-2] for value in row]
print(category_colnames);

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [21]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head();
categories.shape[0]

26180

### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [22]:
for column in categories:
    # set each value to be the last character of the string
    categories[column] = [value[-1] for value in categories[column]]

    # convert column from string to numeric
    categories[column] = pd.to_numeric(categories[column])
categories.head();

### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [23]:
# drop the original categories column from `df`
df = df_Initial.drop(columns=['categories'])
df.head();

In [24]:
# concatenate the original dataframe with the new `categories` dataframe
df = pd.concat([df, categories], axis=1)
print(df.shape[0],df.head())


26248      id                                            message  \
0   2.0  Weather update - a cold front from Cuba that c...   
1   7.0            Is the Hurricane over or is it not over   
2   8.0                    Looking for someone but no name   
3   9.0  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12.0  says: west side of Haiti, rest of the country ...   

                                            original   genre  related  \
0  Un front froid se retrouve sur Cuba ce matin. ...  direct      1.0   
1                 Cyclone nan fini osinon li pa fini  direct      1.0   
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct      1.0   
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct      1.0   
4  facade ouest d Haiti et le reste du pays aujou...  direct      1.0   

   request  offer  aid_related  medical_help  medical_products  ...  \
0      0.0    0.0          0.0           0.0               0.0  ...   
1      0.0    0.0          1.0          

### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [25]:
# check number of duplicates
num_duplicates = df.duplicated().sum()

print("Number of duplicates:", num_duplicates)

Number of duplicates: 41


In [26]:
# drop duplicates
df = df.drop_duplicates()

# convert to dataframe
df = pd.DataFrame(df)

In [27]:
# check number of duplicates
num_duplicates2 = df.duplicated().sum()

print("Number of duplicates (again):", num_duplicates2)
# print(df.shape[0],df.head())
df.head();

Number of duplicates (again): 0


### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [29]:
from sqlalchemy import create_engine

# Create an SQLAlchemy engine to connect to the database
engine = create_engine('sqlite:///MessagesDump.db')

# Write the DataFrame to a SQL database table named 'D_Messages'
df.to_sql('D_Messages', engine, index=False, if_exists='replace')


### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.